In [19]:
import imports
import importlib
importlib.reload(imports)
from imports import *

# https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html
# http://opencorpora.org/dict.php?act=gram
# https://github.com/pymorphy2/pymorphy2/blob/92d546f042ff14601376d3646242908d5ab786c1/pymorphy2/tagset.py#L130
feature_tags_array = [
    OpencorporaTag.PARTS_OF_SPEECH, # часть речи
    OpencorporaTag.GENDERS, # род
    OpencorporaTag.NUMBERS, # число
    OpencorporaTag.CASES, # падеж
    OpencorporaTag.ASPECTS, # соверш / несоверш
    OpencorporaTag.TRANSITIVITY, # перех / непереходный
    OpencorporaTag.PERSONS, # лицо
    OpencorporaTag.TENSES, # время
    OpencorporaTag.MOODS, # наклонение
    OpencorporaTag.VOICES, # залог
    #INVOLVEMENT
    ['Prnt'], # вводные слова
    ['Apro'], # местоимение
    ['NUMB'], # число вида 1234
    ['LATIN'], # текст на английском
    ['UNKN'], # неизвестный токен
    ['PUNCT_DASH', 'PUNCT_DOT', 'PUNCT_COMMA', 'PUNCT_QUOTE',
     'PUNCT_LEFT_PARENTHESIS', 'PUNCT_RIGHT_PARENTHESIS' ], # "()
    ['CAPITALIZED'], # начинается с заглавной буквы
    ['Fixd', 'Abbr'] # неизменяемое, сокращение
]

CUT_NAVEC_TAGS_ARRAY = [
    #'NOUN', #'ADJF'
]

params = build_params({
    "VARIANTS_CNT": 1,
    "TARGET_CLASSES_COUNT": 3,
    "INPUT_WORDS_CNT": 16,
    "feature_tags_array": feature_tags_array,
    "PUNCTUATION_TARGET": {
        "$empty": NO_PUNCT,
        ",": 1,
        ".": 2,
        "!": 2,
        "?": 2,
    },
    "USE_NAVEC": True,
    'CUT_NAVEC_TAGS_SET': set(CUT_NAVEC_TAGS_ARRAY),
    'INFECTED_TEXT_PROB': 0.1,
    "RETAIN_LEFT_PUNCT": True,
    'type': 'lenta',
})

In [102]:
from remote_server import RemoteRunnerServer

try:
    def func(): return 1
    assert server.rpc_simple(func) == 1
    print("server already running")
except:
    server = RemoteRunnerServer()
    server.run()

def server_install_packages():
    def package_install(package_data):
        global egg_file
        import tempfile
        import pkgutil
        import importlib
        import sys
        egg_file = tempfile.NamedTemporaryFile(suffix=".egg")
        egg_file.write(package_data)
        egg_file.flush()

        if egg_file.name not in sys.path:
            sys.path.insert(0, egg_file.name)
        for i in list(pkgutil.iter_modules([egg_file.name])):
            importlib.reload(importlib.import_module(i.name))
            print("loaded ", i.name)

    !python setup.py bdist_egg > /dev/null
    with open(glob.glob('dist/*.egg')[0], "rb") as f:
        package_data = f.read()
    server.rpc_simple(package_install, package_data)
    server.rpc_simple(exec, "from imports import *")
    # server.rpc_simple(exec, "import importlib\nimport dataset_builder\nimportlib.reload(dataset_builder)")

server_install_packages()

server already running
/home/misha-sh/micromamba/envs/pytorch-env/envs/pytorch-env/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
zip_safe flag not set; analyzing archive contents...


In [103]:
server_install_packages()
server.rpc_simple(dataset_builder.create_dataset, ['а, б'], params, False)

/home/misha-sh/micromamba/envs/pytorch-env/envs/pytorch-env/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
zip_safe flag not set; analyzing archive contents...


(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]], dtype=torch.float16),
 tensor([[0, 1, 0],
         [1, 0, 0]], dtype=torch.int32),
 [['PAD',
   'PAD',
   'PAD',
   'PAD',
   'PAD',
   'PAD',
   'PAD',
   'а',
   ' #,# ',
   'б',
   'PAD',
   'PAD',
   'PAD',
   'PAD',
   'PAD',
   'PAD',
   'PAD'],
  ['PAD',
   'PAD',
   'PAD',
   'PAD',
   'PAD',
   'PAD',
   'а',
   ',',
   ' #$empty# ',
   'б',
   'PAD',
   'PAD',
   'PAD',
   'PAD',
   'PAD',
   'PAD',
   'PAD']],
 tensor([False, False]))

(1048576, 4)

In [85]:
import dill
print(size_of_tensor(torch.rand((1024, 1024, 1024)).half()) / 1024 / 1024, "MB")
print(len(dill.dumps(torch.rand((1024, 1024, 1024)))) / 1024 / 1024, "MB")

2048.0 MB
4096.000408172607 MB


In [13]:
import distributed
distributed

<module 'distributed' from '/home/misha-sh/micromamba/envs/pytorch-env/lib/python3.10/site-packages/distributed/__init__.py'>

In [10]:
import dill
import pymorphy3
dill.dumps(pymorphy3, byref=False)

b'\x80\x04\x951\x00\x00\x00\x00\x00\x00\x00\x8c\ndill._dill\x94\x8c\x0e_import_module\x94\x93\x94\x8c\tpymorphy3\x94\x85\x94R\x94.'

In [8]:
import torch
def func(x):
    return torch.Tensor(10) # x * 2 #torch.Tensor((1000, 1000, 1000))

server.send_globals(globals())# get_sendable_globals(globals()))
server.rpc_simple(func, 100)

tensor([-9.8635e-22,  4.5646e-41, -9.8635e-22,  4.5646e-41,  4.4842e-44,
         0.0000e+00,  1.1210e-43,  0.0000e+00,  1.3183e-27,  3.0882e-41])

In [51]:

   
import dill
dill.loads(dill.dumps(get_sendable_globals(globals())))


{'torch': <module 'torch' from '/home/misha-sh/micromamba/envs/pytorch-env/lib/python3.10/site-packages/torch/__init__.py'>,
 'func': <function __main__.func(x)>,
 '_3': <module 'torch' from '/home/misha-sh/micromamba/envs/pytorch-env/lib/python3.10/site-packages/torch/__init__.py'>,
 '_5': <module 'torch' from '/home/misha-sh/micromamba/envs/pytorch-env/lib/python3.10/site-packages/torch/__init__.py'>,
 '_6': module,
 'is_module': <function __main__.is_module(x)>,
 'get_modules_dict': <function __main__.get_modules_dict(globs)>,
 'dill': <module 'dill' from '/home/misha-sh/micromamba/envs/pytorch-env/lib/python3.10/site-packages/dill/__init__.py'>,
 '_35': <module 'dill' from '/home/misha-sh/micromamba/envs/pytorch-env/lib/python3.10/site-packages/dill/__init__.py'>,
 'is_sendable': <function __main__.is_sendable(x)>,
 'get_sendable_globals': <function __main__.get_sendable_globals(globs)>}

In [33]:
get_modules_dict(globals())

{'torch': <module 'torch' from '/home/misha-sh/micromamba/envs/pytorch-env/lib/python3.10/site-packages/torch/__init__.py'>,
 '_3': <module 'torch' from '/home/misha-sh/micromamba/envs/pytorch-env/lib/python3.10/site-packages/torch/__init__.py'>,
 '_5': <module 'torch' from '/home/misha-sh/micromamba/envs/pytorch-env/lib/python3.10/site-packages/torch/__init__.py'>,
 'dill': <module 'dill' from '/home/misha-sh/micromamba/envs/pytorch-env/lib/python3.10/site-packages/dill/__init__.py'>}

In [97]:
torch.Tensor([1, 2, 3]).numpy().save()

array([1., 2., 3.], dtype=float32)

In [55]:
a.seek(0)

0

In [56]:
a.read(3)

b'\x80\x04\x95'

In [57]:
a.read(8)

b'\xc7\x9d\x00\x00\x00\x00\x00\x00'

In [79]:
import io

# class Kek(io.RawIOBase):
#     def __init__(self, a):
#         self.a = a
    
#     def read(self, cnt=-1): 
#         print(cnt)
#         return self.a.read(cnt)
#     def readline(self): raise Exception("no realization")
#     def readable(self): return True
#     def seekable(self): return True
#     def seek(self, byte, whence=0):
#         print(byte, whence)
#         self.a = io.BytesIO(self.a_copy)
#         return 0
#     def tell(self):
#         print("tell")
#         return 0

class Kek(io.BytesIO):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def read(self, cnt=-1):
        print("read", cnt)
        return super().read(cnt)
    
    def readinto(self, b):
        print("readitno", len(b))
        return super().readinto(b)
    

a = io.BytesIO()
dill.dump(torch.rand((100 * 1000)), a)
dill.dump(torch.rand((100, 100)), a)

a.seek(0)

a = Kek(a.read())

dill.load(a)
print(" ")
dill.load(a)

read 1
read 1
read 1
read 8
read 76
read 1
read 4
readitno 400254
read 1
read 8
read 57
 
read 1
read 1
read 1
read 8
read 40391


tensor([[0.0343, 0.3130, 0.9664,  ..., 0.9170, 0.8214, 0.4181],
        [0.1939, 0.8381, 0.9809,  ..., 0.8195, 0.0140, 0.3864],
        [0.5518, 0.9054, 0.8290,  ..., 0.7422, 0.3867, 0.7458],
        ...,
        [0.3726, 0.8367, 0.0867,  ..., 0.4286, 0.4905, 0.2414],
        [0.8013, 0.9115, 0.5852,  ..., 0.7848, 0.8943, 0.6425],
        [0.7150, 0.8926, 0.2941,  ..., 0.7563, 0.1146, 0.6488]])

In [83]:
import mmap

with open('input.bin', 'rb') as f:
    with mmap.mmap(f.fileno(), length=0, access=mmap.ACCESS_READ) as mm:
        memoryview(mm)

TypeError: memoryview: a bytes-like object is required, not 'Kek'

In [ ]:
from enum import Enum

# class FRAME(Enum):
#     FUNCTION = 1,
#     ARGS = 2,
#     KWARGS = 3,
#     TENSOR = 4

# def decode_frame(f):
#     typ = int.from_bytes(f.read(4), byteorder='little')
#     decoded = dill.load(f)
#     #     decoded = torch.load(io.BytesIO(f.read(size))
#     return FRAME(typ), decoded
    
# def send_func(f, func)
#     f.write(FRAME.FUNCTION)
#     dill.dump(func, f)
# def send_args(f, args):
#     f.write(FRAME.ARGS)
#     dill.dump(func, f)
# def send_kwargs(f, args):
#     f.write(FRAME.KWARGS)
#     dill.dump(func, f)
# def send_tensor(f, tensor):
#     f.write(FRAME.TENSOR)
#     torch.save(tensor, f)

    